In [ ]:
#!pip install kafka
from confluent_kafka import Consumer, KafkaException
import json

# Configuração do Kafka Consumer
consumer_conf = {
    'bootstrap.servers': 'kafka:9092',  # Endereço do Kafka
    'group.id': 'consumer-group-1',     # Nome do grupo de consumidores
    'auto.offset.reset': 'earliest'    # Inicia do começo do tópico se não houver commits
}

# Criar o consumidor
consumer = Consumer(consumer_conf)
topic = 'kf-topic'
consumer.subscribe([topic])

print(f"Consumidor Kafka conectado e escutando o tópico '{topic}'...")

try:
    while True:
        msg = consumer.poll(1.0)  # Aguarda mensagem por até 1 segundo
        if msg is None:
            continue
        if msg.error():
            if msg.error().code() == KafkaException._PARTITION_EOF:
                # Fim de partição
                continue
            else:
                raise KafkaException(msg.error())
        # Decodificar a mensagem
        try:
            payload = json.loads(msg.value().decode('utf-8'))
            print(f"Mensagem recebida: {payload}")
        except json.JSONDecodeError as e:
            print(f"Erro ao decodificar mensagem: {msg.value()}. Detalhes: {e}")
except KeyboardInterrupt:
    print("\nEncerrando o consumidor...")
finally:
    # Fechar o consumidor ao sair
    consumer.close()
